<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/label_export/images.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/label_export/images.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

How to export data, with examples for each type of export along with details on optional parameters and filters.

# Image Data Export

In [ ]:
!pip install "labelbox[data]" -q

In [ ]:
import labelbox as lb

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = "API KEY here"
client = lb.Client(api_key=API_KEY)

# Export data rows from a project

When you export data rows from a project, you can narrow down your data rows by last_activity_at, label_created_at, and data_row_ids. Then, when you export from a project, you may choose to include or exclude certain attributes in your export.

You can view the JSON export formats for each data type here: https://docs.labelbox.com/reference/label-export#export-specifications

In [ ]:
# Set the export params to include/exclude certain fields. Make sure each of these fields are correctly grabbed 
export_params= {
  "attachments": True,
  "metadata_fields": True,
  "data_row_details": True,
  "project_details": True,
  "performance_details": True
}

# You can set the range for last_activity_at and label_created_at in one these formats: 
# YYYY-MM-DD or YYYY-MM-DD hh:mm:ss or ISO 8601 format which is YYYY-MM-DDThh:mm:ss±hhmm
# The ISO 8061 allows you to specify the timezone, but the other two formats assume timezone from the user's workspace settings.
# You can also set a list of data row ids to export. 
# For context, last_activity_at captures the creation and modification of labels, metadata, status, comments and reviews.

# Note: This is an AND logic between the filters, so usually using one filter is sufficient.
filters= {
  "last_activity_at": ["2000-01-01 00:00:00", "2050-01-01 00:00:00"],
  "label_created_at": ["2000-01-01 00:00:00", "2050-01-01 00:00:00"],
  "data_row_ids": ["data_row_id_1", "data_row_id_2"] 
}

project = client.get_project("Project ID")
export_task = project.export_v2(params=export_params, filters=filters)
export_task.wait_till_done()

if export_task.errors:
  print(export_task.errors)

export_json = export_task.result
print("results: ", export_json)

# Export from a dataset

You can export data rows and all their information from a Dataset or a Catalog Slice.

When exporting from Catalog, you can include information about a data row from all projects and model runs to which it belongs. Specifically, for the selected data rows, you can export the labels from multiple projects and/or the predictions from multiple model runs.

As shown below, the project_ids and model_run_ids parameters accept a list of IDs

In [ ]:
# Set the export params to include/exclude certain fields. Make sure each of these fields are correctly grabbed 
export_params= {
    "attachments": True,
    "metadata_fields": True,
    "data_row_details": True,
    "project_details": True,
    "performance_details": True,
    "project_ids": ["<project_id_1>", "<project_id_2>"],
    "model_run_ids": ["<model_run_id_1>", "<model_run_id_2>"],
  
}

# You can set the range for last_activity_at
# For context, last_activity_at captures the creation and modification of labels, metadata, status, comments and reviews.
# Note: This is an AND logic between the filters, so usually using one filter is sufficient.

filters= {
            "last_activity_at": ["2000-01-01 00:00:00", "2050-01-01 00:00:00"]
}

dataset = client.get_dataset("<dataset_id>")
export_task = dataset.export_v2(params=export_params, filters=filters)
export_task.wait_till_done()
if export_task.errors:
  print(export_task.errors)
export_json = export_task.result
print("results: ", export_json)

# Export from a slice

In [ ]:
# Set the export params to include/exclude certain fields. Make sure each of these fields are correctly grabbed 
export_params= {
    "attachments": True,
    "metadata_fields": True,
    "data_row_details": True,
    "project_details": True,
    "performance_details": True,
    "project_ids": ["<project_id_1>", "<project_id_2>"],
    "model_run_ids": ["<model_run_id_1>", "<model_run_id_2>"]
}

# There is no filter parameter in export_v2() for slice

catalog_slice = client.get_catalog_slice("<slice_id>")
export_task = catalog_slice.export_v2(params=export_params)
export_task.wait_till_done()
if export_task.errors:
  print(export_task.errors)
export_json = export_task.result
print("results: ", export_json)

# Export data rows from a model run

In [ ]:
# Set the export params to include/exclude certain fields. Make sure each of these fields are correctly grabbed 
export_params= {
    "attachments": True,
    "metadata_fields": True,
    "data_row_details": True,
    "predictions": True,
}

model_run = client.get_model_run("Enter Model run ID")
export_task = model_run.export_v2(params=export_params)
export_task.wait_till_done()
print(export_task.errors)
export_json = export_task.result
print("results: ", export_json)